# Machine learning tests are run here
Some code blocks always have to be run, some can be skipped if only certain tests are of interest.

# Imports (mandatory)
We start with importing the necessary packages.

In [85]:
import numpy as np
import tensorflow as tf
from sklearn import svm

# Make the dataset (mandatory)
Here, we make the dataset that we will use.

In [84]:
# load the features
dftrain = np.load('features.npy')
dftrain = dftrain[:,17:135] # discard some peripheral patients and the chin landmarks
# load the corresponding labels
y_train = np.load('labels.npy')
# y_train = y_train[50:202] # discard the same peripheral patients

dftrain = dftrain / 900.0

classes = ['peripheral palsy', 'central palsy', 'healthy']


# Print the dataset (optional)
See the dataset that we're working with.

In [83]:
# print("Features:", dftrain[0])
# print("Labels:", classes[y_train[0]])
print("Features shape:",dftrain.shape)
print("Labels shape:",y_train.shape)

Features shape: (203, 118)
Labels shape: (203,)


# Model 1 (optional)
This is the first experiment. A DNN is made with many different amounts of hidden layers and nodes. The final accuracy is printed using LOOCV.

In [82]:
# use leave-one-out cross-validation to test the accuracy of the model
correct = 0
n = len(y_train)
for i in range(len(y_train)):
    # print("Run", i, "of", len(dftrain))
    x_test = dftrain[i]
    x_test = np.reshape(x_test, (1,-1))
    y_test = y_train[i]
    x_train_loocv = np.delete(dftrain, i, 0)
    y_train_loocv = np.delete(y_train, i, 0)
    firstmodel = tf.keras.Sequential([
        # tf.keras.layers.Dense(136, activation='relu'),
        tf.keras.layers.Dense(108800, activation='relu'),
        # tf.keras.layers.Dense(10880, activation='relu'),
        tf.keras.layers.Dense(3, activation='sigmoid'),
    ])

    firstmodel.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics = tf.keras.metrics.CategoricalAccuracy(),
    )
    firstmodel.fit(x_train_loocv, y_train_loocv, epochs=1, verbose=0, batch_size=len(x_train_loocv), shuffle=True)
    prediction = np.argmax(firstmodel.predict(x_test))
    # print("Prediction:", classes[prediction.astype(int)])
    # print("Actual:    ", classes[y_test.astype(int)])
    if (prediction == y_test):
        correct += 1
    #     print("Prediction is correct!")
    # else:
    #     print("Prediction not correct.")
    n += 1
print("Final accuracy =", correct/n)

Final accuracy = 0.5073891625615764


# Model 2 (optional)
This is the second experiment. A support vector machine (SVM) will be built.

In [80]:
# use leave-one-out cross-validation to test the accuracy of the model
correct = 0
n = len(y_train)
for i in range(len(y_train)):
    x_test = dftrain[i]
    x_test = np.reshape(x_test, ([1,-1]))
    y_test = y_train[i]
    x_train_loocv = np.delete(dftrain, i, 0)
    y_train_loocv = np.delete(y_train, i, 0)
    model2 = svm.SVC(kernel='poly',degree=5)
    model2.fit(x_train_loocv, y_train_loocv)
    prediction = model2.predict(x_test)
    if (prediction == y_test):
        correct += 1
print("Final accuracy =", correct/n)

Final accuracy = 0.8078817733990148
